In [52]:
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)
import pandas as pd
import logging
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pprint
import matplotlib.pyplot as plt
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [49]:
def get_f1_score(eval_df,model_outputs):
    predictions = []
    for x in model_outputs:
        predictions.append(np.argmax(x))
    f1 = f1_score(eval_df['labels'], predictions)
    return f1

In [7]:
seed = 42

In [22]:
df = pd.read_csv('ementa_acordao_rte.csv')
df.head()

,ementa,acordao,entail
0,LEI – APLICAÇÃO NO TEMPO – TRIBUTO – IRRETROAT...,"Vistos, relatados e discutidos estes autos, ac...",1
1,IMUNIDADE – LIVROS FISCAIS. O fato de a pessoa...,"Vistos, relatados e discutidos estes autos, ac...",1
2,DEVIDO PROCESSO LEGAL – SITUAÇÃO CONSTITUÍDA –...,"Vistos, relatados e discutidos estes autos, ac...",1
3,SERVIÇO PÚBLICO – DESLIGAMENTO – DEVIDO PROCES...,"Vistos, relatados e discutidos estes autos, ac...",1
4,EMENTA: RECURSO EXTRAORDINÁRIO. CONSTITUCIONAL...,"Vistos, relatados e discutidos estes autos, ac...",1


In [23]:
X = []
y = []
for ementa,acordao,label in df.values:
  X.append([ementa,acordao,label])
  y.append(label) 

In [21]:
print('total samples:',len(X))

total samples: 858


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed)
train_data = []
eval_data = []
for example in X_train:
  train_data.append(example)
for example in X_test:
  eval_data.append(example)
print('train size:',len(train_data),'test size:',len(eval_data))

train size: 686 test size: 172


In [25]:
train_df = pd.DataFrame(train_data)
train_df.columns = ["text_a", "text_b", "labels"]
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text_a", "text_b", "labels"]

In [105]:
eval_df['labels'].value_counts()

1    102
0     70
Name: labels, dtype: int64

In [ ]:
eval_df['labels'].hist()
plt.show()

In [104]:
train_df['labels'].value_counts()

1    408
0    278
Name: labels, dtype: int64

In [ ]:
train_df['labels'].hist()
plt.show()

In [ ]:
def get_args(output_d):
    model_args = ClassificationArgs(num_train_epochs=2,use_multiprocessing = False,
                                use_multiprocessing_for_evaluation = False,
                                overwrite_output_dir = True,
                                manual_seed=seed,
                                output_dir=output_d,
                                fp16=False)
    return model_args

In [119]:
models = []

In [120]:
output_dir = 'model_outputs/legal_bERTimbau_sts_large_output'
!rm -r $output_dir
model_args = get_args(output_dir)
name = 'Legal-BERTimbau-sts-large-ma-v3'
model = ClassificationModel(
    "bert", "Legal-BERTimbau-sts-large-ma-v3", args=model_args,use_cuda=True, ignore_mismatched_sizes=True
)
models.append((model,name))

rm: cannot remove 'model_outputs/legal_bERTimbau_sts_large_output': No such file or directory


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Legal-BERTimbau-sts-large-ma-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [121]:
output_dir = 'model_outputs/legal_bERTimbau_large_output'
!rm -r $output_dir
model_args = get_args(output_dir)
name = 'Legal-BERTimbau-large-TSDAE-v5'
model = ClassificationModel(
    "bert", "Legal-BERTimbau-large-TSDAE-v5", args=model_args,use_cuda=True, ignore_mismatched_sizes=True
)
models.append((model,name))

rm: cannot remove 'model_outputs/legal_bERTimbau_large_output': No such file or directory


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Legal-BERTimbau-large-TSDAE-v5 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [122]:
output_dir = 'model_outputs/legal_bERTimbau_base_output'
!rm -r $output_dir
model_args = get_args(output_dir)
name = 'Legal-BERTimbau-base-TSDAE'
model = ClassificationModel(
    "bert", "Legal-BERTimbau-base-TSDAE", args=model_args,use_cuda=True, ignore_mismatched_sizes=True
)
models.append((model,name))

rm: cannot remove 'model_outputs/legal_bERTimbau_base_output': No such file or directory


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Legal-BERTimbau-base-TSDAE and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [123]:
output_dir = 'model_outputs/court_decisions_output'
!rm -r $output_dir
model_args = get_args(output_dir)
name = 'bertimbau-base-finetuned-brazilian_court_decisions'
model = ClassificationModel(
    "bert", "bertimbau-base-finetuned-brazilian_court_decisions", args=model_args,use_cuda=True, ignore_mismatched_sizes=True
)
models.append((model,name))

rm: cannot remove 'model_outputs/court_decisions_output': No such file or directory


In [124]:
output_dir = 'model_outputs/base_portuguese_output'
!rm -r $output_dir
model_args = get_args(output_dir)
name = 'bert-base-portuguese-cased'
model = ClassificationModel(
    "bert", "bert-base-portuguese-cased", args=model_args,use_cuda=True, ignore_mismatched_sizes=True,
)
models.append((model,name))

rm: cannot remove 'model_outputs/base_portuguese_output': No such file or directory


Some weights of the model checkpoint at bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model chec

In [125]:
output_dir = 'model_outputs/base_multilingual_output'
!rm -r $output_dir
model_args = get_args(output_dir)
name = 'bert-base-multilingual-cased'
model = ClassificationModel(
    "bert", "bert-base-multilingual-cased", args=model_args,use_cuda=True, ignore_mismatched_sizes=True,
)
models.append((model,name))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [126]:
trained_models = []
for model, name in models:
    print(f'training model {name}:')
    model.train_model(train_df)
    trained_models.append((model,name))

training model Legal-BERTimbau-sts-large-ma-v3:


INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_2_3


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:   Starting fine-tuning.


Running Epoch 0 of 2:   0%|          | 0/86 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:   Starting fine-tuning.


Running Epoch 0 of 2:   0%|          | 0/86 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/86 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to model_outputs/base_multilingual_output.


In [127]:
model_results = []
for model,name in trained_models:
    result, model_outputs, wrong_predictions = model.eval_model(
    eval_df
)
    f1 = get_f1_score(eval_df,model_outputs)
    result['f1-score'] = f1
    model_results.append((result,name))

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_3


Running Evaluation:   0%|          | 0/22 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.40187543791837627, 'tp': 81, 'tn': 42, 'fp': 28, 'fn': 21, 'auroc': 0.8004201680672268, 'auprc': 0.8610616607656268, 'eval_loss': 0.6266042390330271}
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_3


Running Evaluation:   0%|          | 0/22 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.379666860786319, 'tp': 74, 'tn': 46, 'fp': 24, 'fn': 28, 'auroc': 0.7438375350140056, 'auprc': 0.8080016654977341, 'eval_loss': 0.5994966077533636}
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_3


...

INFO:simpletransformers.classification.classification_model:{'mcc': 0.6844205247457985, 'eval_loss': 0.38027972457083786}
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_3


Running Evaluation:   0%|          | 0/22 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.8722437173187451, 'tp': 102, 'tn': 59, 'fp': 11, 'fn': 0, 'auroc': 0.9565826330532212, 'auprc': 0.9592621015204901, 'eval_loss': 0.22064606938511133}
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_3


Running Evaluation:   0%|          | 0/22 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.4860940847675411, 'tp': 91, 'tn': 39, 'fp': 31, 'fn': 11, 'auroc': 0.780672268907563, 'auprc': 0.8283721524280665, 'eval_loss': 0.5758592323823408}


In [128]:
for result,name in model_results:
    r = pprint.pformat(result, indent=2)
    print(f'Modelo: {name}\nResultados: \n{r}')

Modelo: Legal-BERTimbau-sts-large-ma-v3
Resultados: 
{ 'auprc': 0.8610616607656268,
  'auroc': 0.8004201680672268,
  'eval_loss': 0.6266042390330271,
  'f1-score': 0.7677725118483412,
  'fn': 21,
  'fp': 28,
  'mcc': 0.40187543791837627,
  'tn': 42,
  'tp': 81}
Modelo: Legal-BERTimbau-large-TSDAE-v5
Resultados: 
{ 'auprc': 0.8080016654977341,
  'auroc': 0.7438375350140056,
  'eval_loss': 0.5994966077533636,
  'f1-score': 0.74,
  'fn': 28,
  'fp': 24,
  'mcc': 0.379666860786319,
  'tn': 46,
  'tp': 74}
Modelo: Legal-BERTimbau-base-TSDAE
Resultados: 
{ 'auprc': 0.9220956915790613,
  'auroc': 0.9301120448179272,
  'eval_loss': 0.32674143548038875,
  'f1-score': 0.9090909090909091,
  'fn': 7,
  'fp': 12,
  'mcc': 0.7699847368322048,
  'tn': 58,
  'tp': 95}
Modelo: bertimbau-base-finetuned-brazilian_court_decisions
Resultados: 
{ 'eval_loss': 0.38027972457083786,
  'f1-score': 0.8773584905660377,
  'mcc': 0.6844205247457985}
Modelo: bert-base-portuguese-cased
Resultados: 
{ 'auprc': 0.95926